In [50]:
import nltk
nltk.download('nps_chat')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
from sklearn.metrics import accuracy_score
import torch
import pandas as pd
from datetime import datetime
import torchvision
from torch.utils.tensorboard import SummaryWriter
from transformers import RobertaTokenizerFast,RobertaTokenizer, RobertaForSequenceClassification,RobertaForMaskedLM, TextClassificationPipeline,TFRobertaModel
import numpy as np
import datasets
import re

[nltk_data] Downloading package nps_chat to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [55]:
train_data, test_data = datasets.load_dataset('dutch_social', split =['train', 'test'])

Reusing dataset dutch_social (C:\Users\Emiel\.cache\huggingface\datasets\dutch_social\dutch_social\1.1.0\4ec8e931ab57f4a4483ad4b418676a45a7f6fec1cf6506da7d99c97259f7e02f)


  0%|          | 0/2 [00:00<?, ?it/s]

In [56]:
train_data = train_data.shuffle()
#.select(range(5000))

In [57]:
def remove_link(text):
    return re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE)

def remove_username(text):
    return re.sub('@[\w]+','',text)

def remove_retweet(text):
    
    return text.replace("RT : ", "")

def augment_text(text):
    
   return remove_retweet(remove_username(remove_link(text)))

In [58]:
train_data

Dataset({
    features: ['full_text', 'text_translation', 'screen_name', 'description', 'desc_translation', 'location', 'weekofyear', 'weekday', 'month', 'year', 'day', 'point_info', 'point', 'latitude', 'longitude', 'altitude', 'province', 'hisco_standard', 'hisco_code', 'industry', 'sentiment_pattern', 'subjective_pattern', 'label'],
    num_rows: 162805
})

In [59]:
posts = nltk.corpus.nps_chat.xml_posts()
X_train = [augment_text(x) for x in train_data['full_text']]
y_train  = train_data['label']

X_test = [augment_text(x) for x in test_data['full_text']]
y_test  = test_data['label']

In [60]:
def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features


featuresets = [(dialogue_act_features(post.text), target) for post, target in list(zip(posts,y))]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train((list(zip([dialogue_act_features(x) for x in X_train],y_train))))
print(nltk.classify.accuracy(classifier, (list(zip([dialogue_act_features(x) for x in X_test],y_test)))))

0.5925960050121618


In [61]:
classifier.show_most_informative_features(50)

Most Informative Features
           contains(top) = True                2 : 0      =    154.7 : 1.0
   contains(succesvolle) = True                2 : 1      =    118.6 : 1.0
       contains(strenge) = True                0 : 2      =    118.1 : 1.0
  contains(middelvinger) = True                0 : 2      =    114.7 : 1.0
             contains(😂) = True                2 : 0      =    113.4 : 1.0
          contains(bus…) = True                2 : 1      =    105.3 : 1.0
    contains(gelijkheid) = True                0 : 1      =    101.7 : 1.0
      contains(dankbaar) = True                2 : 1      =     99.4 : 1.0
     contains(nachtrust) = True                2 : 1      =     98.0 : 1.0
        contains(aanta…) = True                0 : 1      =     97.7 : 1.0
             contains(😡) = True                0 : 1      =     97.0 : 1.0
          contains(kben) = True                0 : 1      =     95.2 : 1.0
     contains(walgelijk) = True                0 : 1      =     94.2 : 1.0

In [64]:
line = "Jij bent echt een idioot"
print(classifier.classify(dialogue_act_features(line)))


0


In [65]:
line = "Goed gedaan vriend"
print(classifier.classify(dialogue_act_features(line)))

2
